## Week 2 Day 2 <center> Sales Agent

### We are going to build a simple sales agent that will:

1. Be able to send email
2. Use different tools created by us and
3. Hand off task


We would be creating SENDGRID_API_KEY using its free account for our agents to be able to send emails

In [3]:
from dotenv import load_dotenv
import os
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [4]:
load_dotenv(override=True)

True

In [ ]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
    to_email = To("ed.donner@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

In [17]:
def send_test_email():
    # Debug: Check if environment variables are loaded
    api_key = os.environ.get('SENDGRID_API_KEY')
    from_email_addr = os.getenv("YAHOO_ID")
    to_email_addr = os.getenv("GMAIL_ID")
    
    print(f"API Key exists: {api_key is not None}")
    # print(f"From email: {from_email_addr}")
    # print(f"To email: {to_email_addr}")
    
    if not api_key:
        raise ValueError("SENDGRID_API_KEY environment variable is not set")
    if not from_email_addr:
        raise ValueError("YAHOO_ID environment variable is not set")
    if not to_email_addr:
        raise ValueError("GMAIL_ID environment variable is not set")
    
    try:
        # Initialize SendGrid client
        sg = sendgrid.SendGridAPIClient(api_key=api_key)
        
        # Create the email message - Fixed: Remove .get() method call
        message = Mail(
            from_email=from_email_addr,  # Can pass string directly
            to_emails=to_email_addr,     # Can pass string directly
            subject="Test email from your AI Agent",
            plain_text_content="This is an important test email from your AI Agent"
        )
        
        # Alternative way using the helper classes (if you prefer):
        # from_email = Email(from_email_addr)
        # to_email = To(to_email_addr)
        # content = Content("text/plain", "This is an important test email from your AI Agent")
        # message = Mail(from_email, to_email, "Test email from your AI Agent", content)
        
        # Send the email
        response = sg.send(message)  # Simplified method call
        
        print(f"Email sent successfully!")
        print(f"Status code: {response.status_code}")
        print(f"Response body: {response.body}")
        print(f"Response headers: {response.headers}")
        
    except Exception as e:
        print(f"Error sending email: {str(e)}")
        if hasattr(e, 'body'):
            print(f"Error body: {e.body}")


send_test_email()

API Key exists: True
Email sent successfully!
Status code: 202
Response body: b''
Response headers: Server: nginx
Date: Wed, 13 Aug 2025 01:08:14 GMT
Content-Length: 0
Connection: close
X-Message-Id: -lvnh03YRjSPMT_ZT1Js_g
Access-Control-Allow-Origin: https://sendgrid.api-docs.io
Access-Control-Allow-Methods: POST
Access-Control-Allow-Headers: Authorization, Content-Type, On-behalf-of, x-sg-elas-acl
Access-Control-Max-Age: 600
X-No-CORS-Reason: https://sendgrid.com/docs/Classroom/Basics/API/cors.html
Strict-Transport-Security: max-age=31536000; includeSubDomains
Content-Security-Policy: frame-ancestors 'none'
Cache-Control: no-cache
X-Content-Type-Options: no-sniff
Referrer-Policy: strict-origin-when-cross-origin




# Agent Workflow

#### Creating multiple sales agent that writes emails

In [18]:
sag1 = "You are a sales agent working at ComplAI,\
a company that provides SaaS tool for SOC 2 compliance and preparing for audits, powered by AI.\
You write professional serious cold emails."

    
sag2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

sag3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."
    

In [22]:
sales_agent1 = Agent(
    name="Sales Agent 1",
    instructions=sag1,
    model="gpt-4o-mini")


sales_agent2 = Agent(
    name="Sales Agent 2",
    instructions=sag2,
    model="gpt-4o-mini")

sales_agent3 = Agent(
    name="Sales Agent 3",
    instructions=sag3,
    model="gpt-4o-mini")

In [ ]:
result = await Runner.run(sales_agent1, input="Write a cold sales email")


"Subject: Simplify Your SOC 2 Compliance Journey with ComplAI\n\nHi [Recipient's Name],\n\nI hope this message finds you well. I'm [Your Name], reaching out from ComplAI, where we specialize in making SOC 2 compliance seamless and efficient through our AI-driven SaaS platform.\n\nAs organizations increasingly focus on data security, ensuring compliance with SOC 2 standards is becoming paramount. Our tool not only simplifies the auditing process but also helps in maintaining ongoing compliance, allowing you to focus on what you do best—growing your business.\n\nKey benefits of using ComplAI include:\n\n- **Automated Documentation:** Streamline the creation and management of required documents.\n- **Real-Time Monitoring:** Gain insights into compliance status and areas of improvement.\n- **AI-Driven Insights:** Leverage advanced analytics to enhance security measures.\n\nI would love the opportunity to discuss how ComplAI can support your compliance initiatives. Are you available for a b

In [ ]:
# UNderstand the below run.streamed implementation instead of Runner.run

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplify Your SOC 2 Compliance with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from ComplAI, where we specialize in streamlining the SOC 2 compliance journey for businesses like yours.

As you may know, achieving SOC 2 compliance can be a daunting process—one that requires significant time and resources. That's where we come in. Our AI-powered SaaS tool automates documentation, prepares your team for audits, and simplifies the entire compliance workflow. 

Here are a few key benefits our clients have experienced:

- **Efficiency**: Reduce compliance preparation time by up to 50%.
- **Scalability**: Easily adapt to evolving regulations and audit requirements.
- **Cost Savings**: Minimize the need for external consulting fees.

I’d love to schedule a brief call to discuss how ComplAI can specifically help [Recipient's Company Name] streamline its SOC 2 compliance process. When would be a convenient time for y